In [1]:
import pandas as pd
pd.options.display.max_columns = 50
import numpy as np
import os

# Preprocessing the newest landuse and synpop to match old formatting
The newest synthetic population and landuse data do not contain the same columns or column names as the old files.  Missing columns are joined from old data and column names are changed where appropriate.

Summaries are produced to compare the old and new synthetic households and populations

MAZ employment is scaled to match the number of workers by county according to ACS PUMS county flows.

## Input Data

In [2]:
preprocess_dir = os.getcwd()
base_model_dir = os.path.abspath(os.path.join(preprocess_dir, '..', '..','..'))

In [3]:
base_model_dir

'F:\\Projects\\Clients\\mtc\\updated_networks\\version_12_landuse_scaling'

#### Landuse

In [4]:
maz_data_old = pd.read_csv(os.path.join(base_model_dir, 'input','landuse','maz_data_old_version_with_all_data.csv'))
maz_data_new = pd.read_csv(os.path.join(base_model_dir, 'input', 'landuse','maz_data_from_box.csv'))
# Model uses the maz_data.csv file
maz_data_processed_location = os.path.join(base_model_dir, 'input', 'landuse', 'maz_data.csv')

In [5]:
pums = pd.read_csv(os.path.join(base_model_dir, r'input\visualizer\data\census\acs_pums_2017_5yr\psam_p06.csv'))

#### Synthetic Population

In [6]:
old_synpop_folder = r'F:\Projects\Clients\mtc\updated_networks\version_11_v4_new_population\input\popsyn'
old_hh = pd.read_csv(os.path.join(old_synpop_folder, 'households.csv'))
old_per = pd.read_csv(os.path.join(old_synpop_folder, 'persons.csv'))

In [7]:
new_synpop_folder = os.path.join(base_model_dir, r'input\popsyn')
new_hh = pd.read_csv(os.path.join(new_synpop_folder, 'synthetic_households.csv'))
new_per = pd.read_csv(os.path.join(new_synpop_folder, 'synthetic_persons.csv'))

In [8]:
# Model uses the households.csv and person.csv files
households_processed_location = os.path.join(new_synpop_folder, 'households.csv')
persons_processed_location = os.path.join(new_synpop_folder, 'persons.csv')

## Landuse Processing

### Adding Missing columns to MAZ data

In [9]:
maz_data_old.columns

Index(['MAZ_ORIGINAL', 'TAZ_ORIGINAL', 'DistID', 'DistName', 'CountyID',
       'CountyName', 'ACRES', 'HH', 'POP', 'ag', 'art_rec', 'constr', 'eat',
       'ed_high', 'ed_k12', 'ed_oth', 'fire', 'gov', 'health', 'hotel', 'info',
       'lease', 'logis', 'man_bio', 'man_lgt', 'man_hvy', 'man_tech', 'natres',
       'prof', 'ret_loc', 'ret_reg', 'serv_bus', 'serv_pers', 'serv_soc',
       'transp', 'util', 'emp_total', 'publicEnrollGradeKto8',
       'privateEnrollGradeKto8', 'publicEnrollGrade9to12',
       'privateEnrollGrade9to12', 'comm_coll_enroll', 'EnrollGradeKto8',
       'EnrollGrade9to12', 'collegeEnroll', 'otherCollegeEnroll',
       'AdultSchEnrl', 'hstallsoth', 'hstallssam', 'dstallsoth', 'dstallssam',
       'mstallsoth', 'mstallssam', 'park_area', 'hparkcost', 'numfreehrs',
       'dparkcost', 'mparkcost', 'ech_dist', 'hch_dist', 'parkarea'],
      dtype='object')

In [10]:
maz_data_new.columns

Index(['MAZ', 'TAZ', 'ACRES', 'HH', 'POP', 'ag', 'art_rec', 'constr', 'eat',
       'ed_high', 'ed_k12', 'ed_oth', 'fire', 'gov', 'health', 'hotel', 'info',
       'lease', 'logis', 'man_bio', 'man_hvy', 'man_lgt', 'man_tech', 'natres',
       'prof', 'ret_loc', 'ret_reg', 'serv_bus', 'serv_pers', 'serv_soc',
       'transp', 'unclass', 'util', 'emp_total', 'publicEnrollGradeKto8',
       'publicEnrollGrade9to12', 'privateEnrollGradeKto8',
       'privateEnrollGrade9to12', 'comm_coll_enroll'],
      dtype='object')

In [11]:
len(maz_data_old.columns)

61

In [12]:
len(maz_data_new.columns)

39

In [13]:
missing_cols = [col for col in maz_data_old.columns if col not in maz_data_new.columns]
missing_cols

['MAZ_ORIGINAL',
 'TAZ_ORIGINAL',
 'DistID',
 'DistName',
 'CountyID',
 'CountyName',
 'EnrollGradeKto8',
 'EnrollGrade9to12',
 'collegeEnroll',
 'otherCollegeEnroll',
 'AdultSchEnrl',
 'hstallsoth',
 'hstallssam',
 'dstallsoth',
 'dstallssam',
 'mstallsoth',
 'mstallssam',
 'park_area',
 'hparkcost',
 'numfreehrs',
 'dparkcost',
 'mparkcost',
 'ech_dist',
 'hch_dist',
 'parkarea']

In [14]:
if 'MAZ_ORIGINAL' not in missing_cols:
    merge_cols = missing_cols + ['MAZ_ORIGINAL']
else:
    merge_cols = missing_cols
maz_data_new_append = pd.merge(maz_data_new, maz_data_old[merge_cols], how='left', left_on='MAZ', right_on='MAZ_ORIGINAL')

In [15]:
maz_data_new_append['EnrollGradeKto8'] = maz_data_new_append['publicEnrollGradeKto8'] + maz_data_new_append['privateEnrollGradeKto8']
maz_data_new_append['publicEnrollGrade9to12'] = maz_data_new_append['publicEnrollGrade9to12'] + maz_data_new_append['privateEnrollGrade9to12']

In [16]:
maz_data_new_append = maz_data_new_append[maz_data_old.columns]

### Scaling Employment to match ACS Worker Location by County

In [18]:
puma_to_county_dict = {
    7500: "San Francisco",
    8100: "San Mateo",
    8500: "Santa Clara",
    100: "Alameda",
    1300: "Contra Costa",
    9500: "Solano",
    5500: "Napa",
    9700: "Sonoma",
    4100: "Marin",
}
pums_workers = pums[pums['ESR'].isin([1,2,4,5]) 
               & pums['POWPUMA'].isin(list(puma_to_county_dict.keys()))
               & (pums['JWTR'] != 11)].copy()
pums_workers['work_county'] = pums_workers['POWPUMA'].astype(int).map(puma_to_county_dict)

In [19]:
def create_pums_vs_landuse_employment_comparison(pums_workers, maz_data):
    pums_employment = pums_workers.groupby('work_county')['PWGTP'].sum().to_frame()
    pums_employment.loc['Total'] = pums_employment.sum()
    pums_employment.columns = ['PUMS work location']
    
    landuse_emply = maz_data.groupby('CountyName')['emp_total'].sum().to_frame()
    landuse_emply.loc['Total'] = landuse_emply.sum()
    landuse_emply.columns = ['landuse_employment']
    landuse_emply.index.name = 'work_county'
    
    employment = pd.merge(pums_employment, landuse_emply, left_index=True, right_index=True)
    # employment['difference'] = employment['landuse_employment'] - pums_employment['PUMS work location']
    employment['scaling_factor'] = employment['PUMS work location'] / employment['landuse_employment']
    return employment

In [20]:
emply_comp_before_scaling = create_pums_vs_landuse_employment_comparison(pums_workers, maz_data_new_append)
print("PUMS vs Landuse employment location before scaling")
emply_comp_before_scaling

PUMS vs Landuse employment location before scaling


,PUMS work location,landuse_employment,scaling_factor
work_county,,,
Alameda,683707,839940,0.813995
Contra Costa,355254,425912,0.834102
Marin,112839,140370,0.803868
Napa,72568,87529,0.829074
San Francisco,677855,652946,1.038149
San Mateo,376592,406482,0.926467
Santa Clara,1013788,1040081,0.974720
Solano,138874,167681,0.828204
Sonoma,210971,243322,0.867044


In [23]:
maz_data_new_append_scaled = maz_data_new_append.copy()

emply_cols = [
    'ag', 'art_rec', 'constr', 'eat', 'ed_high', 'ed_k12', 'ed_oth',
    'fire', 'gov', 'health', 'hotel', 'info', 'lease', 'logis', 'man_bio',
    'man_lgt', 'man_hvy', 'man_tech', 'natres', 'prof', 'ret_loc', 'ret_reg',
    'serv_bus', 'serv_pers', 'serv_soc', 'transp', 'util', 'emp_total'
]

# scaling employment by county
for county in emply_comp_before_scaling.index:
    county_filter = (maz_data_new_append_scaled['CountyName'] == county)
    scaling_factor = emply_comp_before_scaling.loc[county, 'scaling_factor']
    print(f"county: {county}, scaling_factor: {round(scaling_factor,2)}, Number of mazs: {county_filter.sum()}")
    maz_data_new_append_scaled.loc[county_filter, emply_cols] = (
        maz_data_new_append_scaled.loc[county_filter, emply_cols] 
        * scaling_factor
    ).round(0).astype(int)

county: Alameda, scaling_factor: 0.81, Number of mazs: 8626
county: Contra Costa, scaling_factor: 0.83, Number of mazs: 5912
county: Marin, scaling_factor: 0.8, Number of mazs: 1418
county: Napa, scaling_factor: 0.83, Number of mazs: 956
county: San Francisco, scaling_factor: 1.04, Number of mazs: 4153
county: San Mateo, scaling_factor: 0.93, Number of mazs: 4454
county: Santa Clara, scaling_factor: 0.97, Number of mazs: 8510
county: Solano, scaling_factor: 0.83, Number of mazs: 2810
county: Sonoma, scaling_factor: 0.87, Number of mazs: 2887
county: Total, scaling_factor: 0.91, Number of mazs: 0


In [24]:
# checking scaling to make sure it worked as expected
emply_comp_after_scaling = create_pums_vs_landuse_employment_comparison(pums_workers, maz_data_new_append_scaled)
print("PUMS vs Landuse employment location after scaling")
emply_comp_after_scaling

PUMS vs Landuse employment location after scaling


,PUMS work location,landuse_employment,scaling_factor
work_county,,,
Alameda,683707,683800,0.999864
Contra Costa,355254,355615,0.998985
Marin,112839,112818,1.000186
Napa,72568,72555,1.000179
San Francisco,677855,677609,1.000363
San Mateo,376592,376638,0.999878
Santa Clara,1013788,1014151,0.999642
Solano,138874,138861,1.000094
Sonoma,210971,211002,0.999853


Scaling factors are very close to 1, but not exactly due to rounding errors introduced when converting scaled employment to integers

In [25]:
maz_data_new_append.to_csv(maz_data_processed_location, index=False)

## Adding Missing columns to Synthetic Population

In [26]:
old_hh.columns

Index(['HHID', 'TAZ', 'MAZ', 'MTCCountyID', 'HHINCADJ', 'NWRKRS_ESR', 'VEH',
       'NP', 'HHT', 'BLD', 'TYPE'],
      dtype='object')

In [27]:
new_hh.columns

Index(['unique_hh_id', 'PUMA', 'TAZ', 'MAZ', 'SERIALNO', 'ADJINC', 'WGTP',
       'NP', 'TYPE', 'BLD', 'HHT', 'HINCP', 'HUPAC', 'NPF', 'TEN', 'VEH',
       'hh_workers_from_esr', 'hh_income_2010'],
      dtype='object')

In [28]:
[col for col in old_hh.columns if col not in new_hh.columns]

['HHID', 'MTCCountyID', 'HHINCADJ', 'NWRKRS_ESR']

In [29]:
old_per.columns

Index(['HHID', 'PERID', 'AGEP', 'SEX', 'SCHL', 'OCCP', 'WKHP', 'WKW',
       'EMPLOYED', 'ESR', 'SCHG'],
      dtype='object')

In [30]:
new_per.columns

Index(['PUMA', 'TAZ', 'MAZ', 'unique_hh_id', 'SERIALNO', 'SPORDER', 'PWGTP',
       'AGEP', 'COW', 'MIL', 'RELP', 'SCHG', 'SCHL', 'SEX', 'WKHP', 'WKW',
       'ESR', 'HISP', 'naicsp07', 'PINCP', 'POWPUMA', 'socp00', 'socp10',
       'indp02', 'indp07', 'occp02', 'occp10', 'occupation', 'employed',
       'employ_status', 'student_status', 'person_type'],
      dtype='object')

In [31]:
[col for col in old_per.columns if col not in new_per.columns]

['HHID', 'PERID', 'OCCP', 'EMPLOYED']

In [32]:
maz_to_county_dict = maz_data_new_append.set_index('MAZ_ORIGINAL')['CountyID'].to_dict()

In [33]:
new_hh['HHID'] = new_hh['unique_hh_id']
new_hh['HHINCADJ'] = new_hh['hh_income_2010'].fillna(0)
new_hh['NWRKRS_ESR'] = new_hh['hh_workers_from_esr']
new_hh['MTCCountyID'] = new_hh['MAZ'].map(maz_to_county_dict)
new_hh['VEH'] = new_hh['VEH'].fillna(-9)
new_hh['HHT'] = new_hh['HHT'].fillna(-9)
new_hh['BLD'] = new_hh['BLD'].fillna(-9)

In [34]:
new_per['HHID'] = new_per['unique_hh_id']
new_per['PERID'] = range(1, len(new_per) + 1)
new_per['OCCP'] = np.where(new_per['occupation'] == 0, 999, new_per['occupation'])
new_per['WKHP'] = new_per['WKHP'].fillna(-9)
new_per['WKW'] = new_per['WKW'].fillna(-9)
new_per['SCHG'] = new_per['SCHG'].fillna(-9)
new_per['SCHL'] = new_per['SCHL'].fillna(-9)
new_per['EMPLOYED'] = new_per['employed']
# new_per['EMPLOYED'] = np.where(new_per['ESR'].isin([1,2,4,5]), 1, 0)

In [35]:
cols_with_na = [col for col in old_hh.columns if any(new_hh[col].isna())]
for col in cols_with_na:
    display(old_per[col].value_counts(dropna=False))
    display(new_per[col].value_counts(dropna=False))
    print("\n")

In [36]:
assert any(new_hh[old_hh.columns].isna()) == True, "Can't have NA values in household table!"
assert any(new_per[old_per.columns].isna()) == True, "Can't have NA values in person table!"

In [37]:
new_hh[old_hh.columns].to_csv(households_processed_location, index=False)
new_per[old_per.columns].to_csv(persons_processed_location, index=False)

## Household Level Summaries

In [38]:
county_dict = {
    1: 'San Francisco',
    2: 'San Mateo',
    3: 'Santa Clara',
    4: 'Alameda',
    5: 'Contra Costa',
    6: 'Solano',
    7: 'Napa',
    8: 'Sonoma',
    9: 'Marin',
}

def calc_hh_stats(hh_df):
    hh_df['county'] = hh_df['MTCCountyID'].map(county_dict)
    stats_df = hh_df.groupby(['county']).agg(
        total_hhs=pd.NamedAgg(column='HHID', aggfunc='count'),
        mean_income=pd.NamedAgg(column='HHINCADJ', aggfunc='mean'),
        total_workers=pd.NamedAgg(column='NWRKRS_ESR', aggfunc='sum'),
        workers_per_hh=pd.NamedAgg(column='NWRKRS_ESR', aggfunc='mean'),
        total_GQ_hhs=pd.NamedAgg(column='TYPE', aggfunc=lambda series: (series > 1).sum()),
        zero_auto_hh=pd.NamedAgg(column='VEH', aggfunc=lambda series: (series == 0).sum()),
        non_zero_auto_hh=pd.NamedAgg(column='VEH', aggfunc=lambda series: (series >= 1).sum()),
    )
    stats_df.loc['Total'] = [stats_df.total_hhs.sum(),
                              hh_df.HHINCADJ.mean(), 
                              stats_df.total_workers.sum(),
                              hh_df.NWRKRS_ESR.mean(),
                              stats_df.total_GQ_hhs.sum(),
                              stats_df.zero_auto_hh.sum(),
                              stats_df.non_zero_auto_hh.sum()]
    
    int_cols = ['total_hhs', 'mean_income', 'total_workers', 'total_GQ_hhs', 'zero_auto_hh', 'non_zero_auto_hh']
    stats_df[int_cols] = stats_df[int_cols].astype(int)
    float_cols = ['workers_per_hh']
    stats_df[float_cols] = stats_df[float_cols].round(2)
    
    return stats_df

new_stats = calc_hh_stats(new_hh)
old_stats = calc_hh_stats(old_hh)

In [39]:
new_stats

,total_hhs,mean_income,total_workers,workers_per_hh,total_GQ_hhs,zero_auto_hh,non_zero_auto_hh
county,,,,,,,
Alameda,577984,90146,806083,1.39,25356,46752,505876
Contra Costa,387760,94987,529253,1.36,4937,17446,365377
Marin,111517,120660,134949,1.21,2810,4191,104516
Napa,52611,89499,73227,1.39,2124,2828,47659
San Francisco,386238,104907,508724,1.32,19670,106435,260133
San Mateo,270802,119841,405121,1.50,5366,12917,252519
Santa Clara,645775,112122,956866,1.48,21719,25253,598803
Solano,144392,74314,196947,1.36,1412,8398,134582
Sonoma,194656,74504,247085,1.27,6344,7218,181094


In [40]:
old_stats

,total_hhs,mean_income,total_workers,workers_per_hh,total_GQ_hhs,zero_auto_hh,non_zero_auto_hh
county,,,,,,,
Alameda,577960,76603,802427,1.39,25367,52168,500425
Contra Costa,387731,88461,528885,1.36,4933,18901,363897
Marin,111518,114727,134854,1.21,2814,4114,104590
Napa,52615,80878,72848,1.38,2129,2980,47506
San Francisco,386257,79701,504332,1.31,19668,122782,243807
San Mateo,270802,111592,404191,1.49,5368,13004,252430
Santa Clara,645680,89605,951058,1.47,21731,30217,593732
Solano,144437,73460,196828,1.36,1413,8695,134329
Sonoma,194638,70848,247083,1.27,6340,7827,180471


In [41]:
def compute_diffs(old_df, new_df):
    for col in old_df.columns:
        comp_df = pd.DataFrame(data={col + '_old' : old_df[col], col + '_new' : new_df[col]})
        comp_df['difference'] = comp_df[col + '_new'] - comp_df[col + '_old']
        if old_df[col].dtype == int:
            comp_df['difference'] = comp_df['difference'].astype(int)
        else:
            comp_df['difference'].round(2)
        comp_df['percent_diff'] = comp_df['difference'] / comp_df[col + '_new'] * 100
        comp_df['percent_diff'] = comp_df['percent_diff'].round(2)
        display(comp_df)
        
compute_diffs(old_stats, new_stats)

,total_hhs_old,total_hhs_new,difference,percent_diff
county,,,,
Alameda,577960,577984,24,0.00
Contra Costa,387731,387760,29,0.01
Marin,111518,111517,-1,-0.00
Napa,52615,52611,-4,-0.01
San Francisco,386257,386238,-19,-0.00
San Mateo,270802,270802,0,0.00
Santa Clara,645680,645775,95,0.01
Solano,144437,144392,-45,-0.03
Sonoma,194638,194656,18,0.01


,mean_income_old,mean_income_new,difference,percent_diff
county,,,,
Alameda,76603,90146,13543,15.02
Contra Costa,88461,94987,6526,6.87
Marin,114727,120660,5933,4.92
Napa,80878,89499,8621,9.63
San Francisco,79701,104907,25206,24.03
San Mateo,111592,119841,8249,6.88
Santa Clara,89605,112122,22517,20.08
Solano,73460,74314,854,1.15
Sonoma,70848,74504,3656,4.91


,total_workers_old,total_workers_new,difference,percent_diff
county,,,,
Alameda,802427,806083,3656,0.45
Contra Costa,528885,529253,368,0.07
Marin,134854,134949,95,0.07
Napa,72848,73227,379,0.52
San Francisco,504332,508724,4392,0.86
San Mateo,404191,405121,930,0.23
Santa Clara,951058,956866,5808,0.61
Solano,196828,196947,119,0.06
Sonoma,247083,247085,2,0.00


,workers_per_hh_old,workers_per_hh_new,difference,percent_diff
county,,,,
Alameda,1.39,1.39,0.00,0.00
Contra Costa,1.36,1.36,0.00,0.00
Marin,1.21,1.21,0.00,0.00
Napa,1.38,1.39,0.01,0.72
San Francisco,1.31,1.32,0.01,0.76
San Mateo,1.49,1.50,0.01,0.67
Santa Clara,1.47,1.48,0.01,0.68
Solano,1.36,1.36,0.00,0.00
Sonoma,1.27,1.27,0.00,0.00


,total_GQ_hhs_old,total_GQ_hhs_new,difference,percent_diff
county,,,,
Alameda,25367,25356,-11,-0.04
Contra Costa,4933,4937,4,0.08
Marin,2814,2810,-4,-0.14
Napa,2129,2124,-5,-0.24
San Francisco,19668,19670,2,0.01
San Mateo,5368,5366,-2,-0.04
Santa Clara,21731,21719,-12,-0.06
Solano,1413,1412,-1,-0.07
Sonoma,6340,6344,4,0.06


,zero_auto_hh_old,zero_auto_hh_new,difference,percent_diff
county,,,,
Alameda,52168,46752,-5416,-11.58
Contra Costa,18901,17446,-1455,-8.34
Marin,4114,4191,77,1.84
Napa,2980,2828,-152,-5.37
San Francisco,122782,106435,-16347,-15.36
San Mateo,13004,12917,-87,-0.67
Santa Clara,30217,25253,-4964,-19.66
Solano,8695,8398,-297,-3.54
Sonoma,7827,7218,-609,-8.44


,non_zero_auto_hh_old,non_zero_auto_hh_new,difference,percent_diff
county,,,,
Alameda,500425,505876,5451,1.08
Contra Costa,363897,365377,1480,0.41
Marin,104590,104516,-74,-0.07
Napa,47506,47659,153,0.32
San Francisco,243807,260133,16326,6.28
San Mateo,252430,252519,89,0.04
Santa Clara,593732,598803,5071,0.85
Solano,134329,134582,253,0.19
Sonoma,180471,181094,623,0.34


## Person Comparisons

In [42]:
new_per_hh = pd.merge(new_per, new_hh, how='left', on='HHID')
old_per_hh = pd.merge(old_per, old_hh, how='left', on='HHID')

In [43]:
def calc_per_stats(hh_df):
    stats_df = hh_df.groupby(['county']).agg(
        total_people=pd.NamedAgg(column='PERID', aggfunc='count'),
        total_students=pd.NamedAgg(column='SCHG', aggfunc=lambda series: (series > 1).sum()),
        total_workers=pd.NamedAgg(column='EMPLOYED', aggfunc='sum'),
    )
    stats_df.loc['Total'] = [stats_df.total_people.sum(),
                              stats_df.total_students.sum(), 
                              stats_df.total_workers.sum()]
    
    stats_df = stats_df.astype(int)
    
    return stats_df

new_per_stats = calc_per_stats(new_per_hh)
old_per_stats = calc_per_stats(old_per_hh)

In [44]:
new_per_stats

,total_people,total_students,total_workers
county,,,
Alameda,1590603,427092,806083
Contra Costa,1122097,292699,529253
Marin,269839,57158,134949
Napa,143311,34531,73227
San Francisco,850563,160419,508724
San Mateo,772955,186817,405121
Santa Clara,1928415,490286,956866
Solano,431959,116718,196947
Sonoma,504149,116992,247085


In [45]:
old_per_stats

,total_people,total_students,total_workers
county,,,
Alameda,1589668,440599,802427
Contra Costa,1121829,292562,528885
Marin,270157,57284,134854
Napa,143250,34824,72848
San Francisco,850173,172576,504332
San Mateo,772677,186355,404191
Santa Clara,1931136,497802,951058
Solano,432013,116943,196828
Sonoma,503798,117093,247083


In [46]:
compute_diffs(old_per_stats, new_per_stats)

,total_people_old,total_people_new,difference,percent_diff
county,,,,
Alameda,1589668,1590603,935,0.06
Contra Costa,1121829,1122097,268,0.02
Marin,270157,269839,-318,-0.12
Napa,143250,143311,61,0.04
San Francisco,850173,850563,390,0.05
San Mateo,772677,772955,278,0.04
Santa Clara,1931136,1928415,-2721,-0.14
Solano,432013,431959,-54,-0.01
Sonoma,503798,504149,351,0.07


,total_students_old,total_students_new,difference,percent_diff
county,,,,
Alameda,440599,427092,-13507,-3.16
Contra Costa,292562,292699,137,0.05
Marin,57284,57158,-126,-0.22
Napa,34824,34531,-293,-0.85
San Francisco,172576,160419,-12157,-7.58
San Mateo,186355,186817,462,0.25
Santa Clara,497802,490286,-7516,-1.53
Solano,116943,116718,-225,-0.19
Sonoma,117093,116992,-101,-0.09


,total_workers_old,total_workers_new,difference,percent_diff
county,,,,
Alameda,802427,806083,3656,0.45
Contra Costa,528885,529253,368,0.07
Marin,134854,134949,95,0.07
Napa,72848,73227,379,0.52
San Francisco,504332,508724,4392,0.86
San Mateo,404191,405121,930,0.23
Santa Clara,951058,956866,5808,0.61
Solano,196828,196947,119,0.06
Sonoma,247083,247085,2,0.00


In [47]:
np.log(1)

0.0